# CPDCTL Demo - ML Model and Python script lifecycle

CPDCTL is a command-line interface (CLI) you can use to manage the lifecycle of models, notebooks, scripts and other assets. By using the assets CLI, you can automate the flow for creating notebooks and scripts and running jobs, moving assets between projects in Watson Studio and deployment spaces.   

This notebook begins by showing you how to install and configure CPDCTL and is then split up into three sections with examples of how to use the commands to:

- Train a model in the project and deploy in the development space 
- Promote assets to the test space and run validation
- Update assets in the production space

## Table of Contents

[1. Install and Configure CPDCTL](#part1)
- [1.1 Installing the latest version of CPDCTL](#part1.1)
- [1.2 Adding CPD cluster configuration settings](#part1.2)

[2. Demo 1: Train a model in the project and deploy in the development space](#part2)
- [2.1 Promote script asset to the space](#part2.1)
- [2.2 Run batch deployment job](#part2.2)

[3. Demo 2: Promote assets to the test space and run validation](#part3)
- [3.1 Export all asssets from the source (DEV) space](#part3.1)
- [3.2 Create test space and import assets there](#part3.2)
- [3.3 Run batch deployment job in the test space](#part3.3)

[4. Demo 3: Update assets in the production space](#part4)
- [4.1 Import new asset versions to the production space](#part4.1)
- [4.2 Updating the existing model deployment](#part4.2)
- [4.3 Run production model deployment validation](#part4.3)

[5. Cleanup](#part5)

## Before you begin
Import the following libraries:

In [1]:
import base64
import json
import os
import requests
import platform
import tarfile
import zipfile
from datetime import datetime
from IPython.core.display import display, HTML

##  1. Installing and configurating CPDCTL <a class="anchor" id="part1"></a>

### 1.1 Installing the latest version of CPDCTL <a class="anchor" id="part1.1"></a>

To use the notebook and environment CLI commands, you need to install CPDCTL. Download the binary from the [CPDCTL GitHub respository](https://github.com/IBM/cpdctl/releases).

Download the binary and then display the version number:

In [2]:
PLATFORM = platform.system().lower()
CPDCTL_ARCH = "{}_amd64".format(PLATFORM)
CPDCTL_RELEASES_URL="https://api.github.com/repos/IBM/cpdctl/releases"
CWD = os.getcwd()
PATH = os.environ['PATH']
CPDCONFIG = os.path.join(CWD, '.cpdctl.config.yml')

response = requests.get(CPDCTL_RELEASES_URL)
assets = response.json()[0]['assets']
platform_asset = next(a for a in assets if CPDCTL_ARCH in a['name'])
cpdctl_url = platform_asset['url']
cpdctl_file_name = platform_asset['name']

response = requests.get(cpdctl_url, headers={'Accept': 'application/octet-stream'})
with open(cpdctl_file_name, 'wb') as f:
    f.write(response.content)
    
display(HTML('<code>cpdctl</code> binary downloaded from: <a href="{}">{}</a>'.format(platform_asset['browser_download_url'], platform_asset['name'])))
display(HTML("<style>div.output_area pre {white-space: pre;}</style>"))
display(HTML("<style>.container { width:90% !important; }</style>"))

In [3]:
%%capture

%env PATH={CWD}:{PATH}
%env CPDCONFIG={CPDCONFIG}

In [4]:
if cpdctl_file_name.endswith('tar.gz'):
    with tarfile.open(cpdctl_file_name, "r:gz") as tar:
        tar.extractall()
elif cpdctl_file_name.endswith('zip'):
    with zipfile.ZipFile(cpdctl_file_name, 'r') as zf:
        zf.extractall()

if CPDCONFIG and os.path.exists(CPDCONFIG):
    os.remove(CPDCONFIG)
    
version_r = ! cpdctl version
CPDCTL_VERSION = version_r.s

os.remove(cpdctl_file_name)

print("cpdctl version: {}".format(CPDCTL_VERSION))

cpdctl version: 1.0.105


### 1.2  Adding CPD cluster configuration settings <a class="anchor" id="part1.2"></a>

Before you can use CPDCTL, you need to add configuration settings. You only need to configure these settings once for the same CPD user and cluster. Begin by entering your IBM Cloud Pak for Data (CPD) credentials and the URL to the CPD cluster:

In [ ]:
CPD_USER_NAME = #'YOUR CPD user name'
CPD_USER_PASSWORD = #'YOUR CPD user password'
CPD_URL = #'YOUR CPD CLUSTER URL'

Add "cpd_user" user to the cpdctl configuration:

In [6]:
! cpdctl config user set cpd_user --username {CPD_USER_NAME} --apikey {CPD_USER_APIKEY}

Add "cpd" cluster to the cpdctl configuration:

In [7]:
! cpdctl config profile set cpd --url {CPD_URL}

Add "cpd" context to the cpdctl configuration:

In [8]:
! cpdctl config context set cpd --profile cpd --user cpd_user

List available contexts:

In [9]:
! cpdctl config context list

Name   Profile   User       Current   
cpd    cpd       cpd_user   *   


and profiles:

In [10]:
! cpdctl config profile list

Name   Type      User   URL   
cpd    private          https://cpd-zen.apps.cpd35rt.os.fyre.ibm.com   


Switch the current context:

In [11]:
! cpdctl config context use cpd

Switched to context "cpd".


List available projects in context:

In [72]:
! cpdctl project list

...
ID                                     Name                               Created                    Description                                          Tags   
47c57769-9876-46bc-ad26-259c559cbc3f   Clustering Demo (git-integrated)   2021-06-21T10:59:35.265Z   Demo showing how to cluster data in CPD using cod…   []   
68d739dd-7c54-41ae-9ed3-f04db182e559   jm-proj                            2021-07-15T10:27:38.452Z                                                        []   
87d3b6e0-19e1-4e4e-8c4c-11f1a9df789c   fvt-project                        2021-06-28T12:46:42.246Z                                                        []   
a5d7960c-5194-4013-9a0c-4da2d3f6583e   test-mo                            2021-06-23T11:43:56.992Z                                                        []   
b9653c0d-8d41-40bb-8bd0-08a44615607a   CPD402-use-case-A                  2021-08-04T14:02:51.279Z                                                        []   
e655370d-4513-45f9-b4c6-c456920447

Choose a project in which you will work:

In [13]:
# You can also specify your project id directly:
project_id = "b9653c0d-8d41-40bb-8bd0-08a44615607a"

## 2. Demo 1: Train a model in the project and deploy in the development space  <a class="anchor" id="part2"></a>

Before starting with this section, please ensure that you have run the cells in [Section 1](#part1) and specified the ID of the project in which you will work.

Suppose you have a notebook created with JupyterLab in Watson Studio, which trains a ML model, and you would like to run the code on a CPD cluster. This section shows how to run the notebook in a project and then promote trained model to a space. 

### 2.1 Promote script asset to the space<a class="anchor" id="part2.1"></a>

List all data assets in your project:

In [14]:
! cpdctl asset search --project-id {project_id} --type-name data_asset --query "*:*"

...
ID                                     Name                         Created                    Description   Type         State       Tags   Size   
9bd1bc72-41d2-40fa-a60c-691816a71e72   credit_risk_training.csv     2021-08-05T09:25:42.000Z                 data_asset   available   []     689622   
b3bb1a37-266f-46ed-9bec-9ffdcee5bca3   credit_risk_regression.csv   2021-08-05T13:09:36.000Z                 data_asset   available   []     694219   


In [15]:
training_data_asset_id = '9bd1bc72-41d2-40fa-a60c-691816a71e72'
regression_data_asset_id = 'b3bb1a37-266f-46ed-9bec-9ffdcee5bca3'

List all notebooks in your project:

In [16]:
! cpdctl asset search --project-id {project_id} --type-name notebook --query "*:*"

...
ID                                     Name          Created                    Description   Type       State       Tags         Size   
aeba686c-a49f-4992-aaf7-0fc5d7a12a7f   train_model   2021-08-05T09:54:57.000Z                 notebook   available   [notebook]   54308   


In [17]:
notebook_id = 'aeba686c-a49f-4992-aaf7-0fc5d7a12a7f'

Check if there is a notebook version. Because of the [known issue #18703](https://github.ibm.com/dap/dap-planning/issues/18703), 
notebooks imported from git do not have versions created.

In [18]:
! cpdctl notebook version list --notebook-id {notebook_id}

...
ID                                     Created   
1562c587-d2bd-485c-a408-3f2f4c2360ec   1628752234913   
94b044bb-afdc-4ba7-a359-c905a5f9167a   1628161224983   
dff4a679-faae-4c79-8bd6-54cbb557c1b7   1628160717665   


In [25]:
! cpdctl notebook version create --notebook-id {notebook_id}

...
              
ID:        1562c587-d2bd-485c-a408-3f2f4c2360ec   
Created:   1628752234913   


Select the runtime environment

In [26]:
! cpdctl environment list --project-id {project_id}

...
ID                                                   Name                Description   
jupconda37oce-b9653c0d-8d41-40bb-8bd0-08a44615607a   jupconda37oce          
jupyterlabpy37oce-b9653c0d-8d41-40bb-8bd0-08a4461…   jupyterlabpy37oce      


In [27]:
env_id = 'jupconda37oce-b9653c0d-8d41-40bb-8bd0-08a44615607a'

MODEL_NAME = 'gcr-scikit-model'

Create a job for the notebook with selected environment

In [36]:
job = {
    "asset_ref": notebook_id,
    "name": "train-gcr-scikit-model-job",
    "configuration": {
        "env_id": env_id,
        "env_variables": [
            "CPD_URL={}".format(CPD_URL),
            "USER_NAME={}".format(CPD_USER_NAME),
            "USER_APIKEY={}".format(CPD_USER_APIKEY),
            "DATA_ASSET_ID={}".format(training_data_asset_id),
            "MODEL_NAME={}".format(MODEL_NAME)
        ]
    }
}
job_json = json.dumps(job)

In [38]:
! cpdctl job create --project-id {project_id} --job '{job_json}'

...
                  
ID:            202edd66-ed1a-4197-b2b9-5512c308216f   
Name:          train-gcr-scikit-model-job   
Created:       2021-08-12T07:14:46Z   
Description:      
Type:          job   
State:         available   
Tags:          []   
Size:          0   
Version:       2   


In [39]:
job_id = '202edd66-ed1a-4197-b2b9-5512c308216f'

In [40]:
job_run = {
    "job_run": {}
}
job_run_json = json.dumps(job_run)

Start the notebook job and wait for results and logs

In [41]:
! cpdctl job run create --project-id {project_id} --job-id {job_id} --job-run '{job_run_json}' --async

...
                  
ID:            b8288e8f-63da-44ec-a6bf-0cdbb9bc284d   
Name:          Notebook Job   
Created:       2021-08-12T07:15:00Z   
Description:      
Type:          job_run   
State:         Starting   
Tags:          []   
Size:          0   
Version:       2   


In [42]:
run_id = 'b8288e8f-63da-44ec-a6bf-0cdbb9bc284d'

In [43]:
! cpdctl job run wait --project-id {project_id} --job-id {job_id} --run-id {run_id}

...
                  
ID:            b8288e8f-63da-44ec-a6bf-0cdbb9bc284d   
Name:          Notebook Job   
Created:       2021-08-12T07:15:00Z   
Description:      
State:         Completed   
Tags:          []   


In [44]:
! cpdctl job run logs --project-id {project_id} --job-id {job_id} --run-id {run_id}

...

Cell 5:

Cell 8:
env: DATA_ASSET_ID=9bd1bc72-41d2-40fa-a60c-691816a71e72

Cell 12:

Cell 15:
Successfully saved data asset content to file: 'training_data.csv'

Cell 19:

Cell 20:
Columns:  ['CheckingStatus', 'LoanDuration', 'CreditHistory', 'LoanPurpose', 'LoanAmount', 'ExistingSavings', 'EmploymentDuration', 'InstallmentPercent', 'Sex', 'OthersOnLoan', 'CurrentResidenceDuration', 'OwnsProperty', 'Age', 'InstallmentPlans', 'Housing', 'ExistingCreditsCount', 'Job', 'Dependents', 'Telephone', 'ForeignWorker', 'Risk']
Number of columns:  21

Cell 22:
Number of records:  5000

Cell 23:

Cell 25:

Cell 40:
0.738047642845244

Cell 44:
Deleting model id 11376def-10a6-49a8-ab03-f300063371b8

--  ----  -------  ----
ID  NAME  CREATED  TYPE
--  ----  -------  ----

Cell 47:
Storing model ...

Done

Cell 48:
------------------------------------  ----------------  ------------------------  -----------------
ID                                    NAME              CREATED                   TYP

Look for the new trained model

In [19]:
! cpdctl asset search --project-id {project_id} --query '*:*' --type-name wml_model

...
ID                                     Name               Created                    Description   Type        State       Tags                  Size   
2c60c3a8-d461-4a66-b375-c3d4bf0c3102   gcr-scikit-model   2021-08-12T07:15:52.000Z                 wml_model   available   [cpdctl-demo-model]   61046   


In [20]:
model_id = '2c60c3a8-d461-4a66-b375-c3d4bf0c3102'

List all spaces and select the development space

In [21]:
! cpdctl space list

...
ID                                     Name                      Created                    Description   State    Tags   
9001e03a-e3b9-4ddf-88db-063651d735b4   qa-space                  2021-06-21T11:11:22.988Z                 active   []   
715f068a-a8c8-4093-a7f2-15ab111c21e5   prod-space                2021-06-21T11:12:12.958Z                 active   []   
a72bfe88-5fc9-4323-90ad-53d94511d120   fvt-space                 2021-06-28T12:50:33.447Z                 active   []   
cdd90e91-792c-4834-a26a-ac979b998fd8   export-space              2021-07-09T12:02:55.754Z                 active   []   
be32f9b8-702f-444e-9902-754464f39853   jm-space                  2021-07-15T10:40:36.849Z                 active   []   
6237d988-c98f-4f70-acc4-c68f34329105   test-space-cli-eb43d624   2021-07-22T07:49:54.127Z                 active   []   
dd383706-f430-4b1a-8ac8-36084fb821f6   test-export               2021-07-28T17:12:57.575Z                 active   []   
48742238-a5c4-41a6-aaef-fa

In [22]:
dev_space_id = '48de138b-afe8-480c-87c3-34f9a9dc1892'

Select trained model and promote to the development space:

In [50]:
import json

promote = {
    "mode": 0,
    "space_id": dev_space_id,
    "metadata": {
        "tags": ["cpdctl-demo", "promoted-asset-{}]".format(model_id)]
    }
}
promote_json = json.dumps(promote)

! cpdctl asset promote --project-id {project_id} --asset-id {model_id} --request-body '{promote_json}'

...
OK


List assets in the development space

In [54]:
! cpdctl asset search --space-id {dev_space_id} --type-name wml_model --query "*:*"

...
ID                                     Name               Created                    Description   Type        State       Tags                                                                 Size   
4baac07a-2f3d-4c7e-9843-45a660ddac1c   gcr-scikit-model   2021-08-12T07:18:17.000Z                 wml_model   available   [cpdctl-demo promoted-asset-2c60c3a8-d461-4a66-b375-c3d4bf0c3102]]   61046   


In [55]:
dev_model_id = '4baac07a-2f3d-4c7e-9843-45a660ddac1c'

Deploy the promoted ML model as a web service (online deployment)

In [56]:
online_json = '{}'
asset_json = json.dumps({"id": dev_model_id})

! cpdctl ml deployment create --space-id {dev_space_id} --asset '{asset_json}' --online '{online_json}'

...
              
ID:        abcc8535-7e22-43cf-9dbf-8ab3f5bfde32   
Name:         
Created:   2021-08-12T07:19:30.422Z   
State:     ready   
Tags:      []   


Promote the regression data set and the evaluation python script

In [54]:
promote = {
    "space_id": dev_space_id
}
promote_json = json.dumps(promote)

! cpdctl asset promote --project-id {project_id} --asset-id {regression_data_asset_id} --request-body '{promote_json}'

...
OK


In [57]:
! cpdctl asset search --project-id {project_id} --type-name script --query "*:*"

...
ID                                     Name             Created                    Description   Type     State       Tags   Size   
9d6575b7-a244-4492-8b0a-aa530dc54c18   evaluate_model   2021-08-05T09:54:57.000Z                 script   available   []     233   


In [58]:
evaluation_script_id = '9d6575b7-a244-4492-8b0a-aa530dc54c18'

Get the python script details

In [59]:
! cpdctl asset get --project-id {project_id} --asset-id {evaluation_script_id} --output json

{
  "attachments": [
    {
      "asset_type": "script",
      "description": "attachment for script",
      "id": "68130b09-5c7b-4148-9bf0-1f409a13fdc7",
      "is_partitioned": false,
      "mime": "application/text",
      "name": "evaluate_model",
      "object_key": "/jupyterlab/evaluate_model.py",
      "test_doc": 0,
      "user_data": {}
    }
  ],
  "entity": {
    "script": {
      "language": {
        "name": "python3"
      },
      "software_spec": {
        "base_id": "c2057dd4-f42c-5f77-a02f-72bdbd3282c9",
        "name": "default_py3.7_opence"
      }
    }
  },
  "metadata": {
    "asset_attributes": [
      "script"
    ],
    "asset_category": "USER",
    "asset_id": "9d6575b7-a244-4492-8b0a-aa530dc54c18",
    "asset_state": "available",
    "asset_type": "script",
    "created_at": "2021-08-05T09:54:57.000Z",
    "description": "",
    "name": "evaluate_model",
    "origin_country": "us",
    "owner_id": "1000330999",
    "rov": {
      "mode": 0
    },
    "size":

In [60]:
! cpdctl environment software-specification list --project-id {project_id} --name "default_py3.7_opence"

...
ID                                     Name                   Created                    Description          Type   
c2057dd4-f42c-5f77-a02f-72bdbd3282c9   default_py3.7_opence   2021-06-17T17:45:09.912Z   Default Python 3.7   software_specification   


Find "default_py3.7_opence" software specification ID

In [38]:
software_specification_name = "default_py3.7_opence"
jmes_query = "resources[0].metadata.asset_id"
result = ! cpdctl environment software-specification list --project-id {project_id} --name '{software_specification_name}' --output json --jmes-query '{jmes_query}' --raw-output
software_specification_id = result.s
print("software specification id: {}".format(software_specification_id))

software specification id: c2057dd4-f42c-5f77-a02f-72bdbd3282c9


Set python script's software spececification

In [48]:
software_spec = {
    "base_id": "{}".format(software_specification_id),
    "name": software_specification_name
}

patch = [{
    "op": "add",
    "path": "/software_spec",
    "value": software_spec
}]
patch_json = json.dumps(patch)

! cpdctl asset attribute update --project-id {project_id} --asset-id {evaluation_script_id} --attribute-key script  --json-patch '{patch_json}'

...
OK


Promote evaluation script to the development space

In [55]:
promote = {
    "space_id": dev_space_id
}
promote_json = json.dumps(promote)

! cpdctl asset promote --project-id {project_id} --asset-id {evaluation_script_id} --request-body '{promote_json}'

...
OK


In [62]:
! cpdctl asset search --space-id {dev_space_id} --query '*:*' --type-name asset

...
ID                                     Name                         Created                    Description   Type         State       Tags                                                 Size   
ee219910-028a-44ff-93a7-00c80b97af47   credit_risk_regression.csv   2021-08-09T13:30:32.000Z                 data_asset   available   []                                                   694219   
882e7ef8-3b28-41e5-9bd6-8d2c9eb4cf6d   evaluate_model               2021-08-09T13:30:54.000Z                 script       available   []                                                   233   
4baac07a-2f3d-4c7e-9843-45a660ddac1c   gcr-scikit-model             2021-08-12T07:18:17.000Z                 wml_model    available   [cpdctl-demo promoted-asset-2c60c3a8-d461-4a66-b3…   61046   


In [23]:
promoted_model_id = '882e7ef8-3b28-41e5-9bd6-8d2c9eb4cf6d'
promoted_regression_data_asset_id = 'ee219910-028a-44ff-93a7-00c80b97af47'

### 2.2 Run batch deployment job <a class="anchor" id="part2.2"></a>

Create batch deployment:

In [93]:
asset = {
    'id': promoted_model_id
}
asset_json = json.dumps(asset)

hardware_spec = {
    'name': 'S'
}
hardware_spec_json = json.dumps(hardware_spec)

batch_json = '{}'

deployment_name = 'model_batch_deployment'

In [94]:
result = ! cpdctl ml deployment create --space-id {dev_space_id} --name '{deployment_name}' --asset '{asset_json}' --hardware-spec '{hardware_spec_json}' --batch '{batch_json}' --output json -j "metadata.id" --raw-output
deployment_id = result.s
print("deployment id: {}".format(deployment_id))

deployment id: bb5a3c0c-fbbd-4a16-a52c-a79feabd7b5e


Create a deployment job

In [95]:
deployment_job_name = 'model_batch_deployment_job'

deployment = {
    'id': deployment_id
}
deployment_json = json.dumps(deployment)

scoring = {
    "input_data_references": [
      {
        "type": "data_asset",
        "connection": {},
        "location": {
          "href": "/v2/assets/{}?space_id={}".format(promoted_regression_data_asset_id, dev_space_id)
        }
      }
    ],
    "output_data_reference": {
      "type": "data_asset",
      "connection": {},
      "location": {
        "name": "evaluation_result.zip"
      }
    }
}
scoring_json = json.dumps(scoring)

In [96]:
result = ! cpdctl ml deployment-job create --space-id {dev_space_id} --name '{deployment_job_name}' --deployment '{deployment_json}' --scoring '{scoring_json}' --output json -j "metadata.id" --raw-output
deployment_job_id = result.s
print('Deployment job ID:', deployment_job_id)

Deployment job ID: bee95e51-a4bb-4599-8b9d-b39b1acd1a7c


In [98]:
result = ! cpdctl ml deployment-job get --space-id {dev_space_id} --job-id {deployment_job_id} --output json
deployment_job = json.loads(result.s)
job_id = deployment_job['entity']['platform_job']['job_id']
run_id = deployment_job['entity']['platform_job']['run_id']

Wait for job completion

In [99]:
! cpdctl job run wait --job-id {job_id} --run-id {run_id} --space-id {dev_space_id}

...
                  
ID:            e126fea4-5f9e-46c9-8d88-5d07840fcb17   
Name:          job run   
Created:       2021-08-09T14:16:51Z   
Description:      
State:         Completed   
Tags:          []   


You can see the batch deployment log:

In [100]:
! cpdctl job run logs --job-id {job_id} --run-id {run_id} --space-id {dev_space_id}

...
{
  "deployment": {
    "id": "bb5a3c0c-fbbd-4a16-a52c-a79feabd7b5e"
  },
  "platform_job": {
    "job_id": "7ec1c5f8-77d5-4afd-a001-e522b924a4ed",
    "run_id": "e126fea4-5f9e-46c9-8d88-5d07840fcb17"
  },
  "scoring": {
    "input_data_references": [
      {
        "connection": {},
        "location": {
          "href": "/v2/assets/ee219910-028a-44ff-93a7-00c80b97af47?space_id=48de138b-afe8-480c-87c3-34f9a9dc1892"
        },
        "type": "data_asset"
      }
    ],
    "output_data_reference": {
      "connection": {},
      "location": {
        "href": "/v2/assets/1ff5fefb-ece3-4ea6-8154-91acd039eb45?space_id=48de138b-afe8-480c-87c3-34f9a9dc1892",
        "name": "evaluation_result.zip"
      },
      "type": "data_asset"
    },
    "status": {
      "completed_at": "2021-08-09T14:17:13.328270Z",
      "running_at": "2021-08-09T14:17:10.079015Z",
      "state": "completed"
    }
  }
}



In [101]:
output_data_asset_id = '1ff5fefb-ece3-4ea6-8154-91acd039eb45'

Clean the job and produced results

In [105]:
! cpdctl job list --space-id {dev_space_id}

...
ID                                     Name                                   Description   Tags   Version   
7ec1c5f8-77d5-4afd-a001-e522b924a4ed   default job - model_batch_deployment                 []     0   


In [103]:
! cpdctl ml deployment delete --space-id {dev_space_id} --deployment-id {deployment_id}

...
OK


In [ ]:
! cpdctl job delete --space-id {dev_space_id} --job-id {deployment_job_id}

In [106]:
! cpdctl asset delete --space-id {dev_space_id} --asset-id {output_data_asset_id}

...
OK


## 3. Demo 2: Promote assets to the test space and run validation<a class="anchor" id="part3"></a>

Before starting with this section, please ensure that you have run the cells in [Section 1](#part1) and [Section 2](#part2).

Having all assets ready in the development space, we can promote (export & import) into the test space. Once there, we can run validation on the regression data set and analyse results.

### 3.1 Export all asssets from the source (DEV) space<a class="anchor" id="part3.1"></a>

List all assets in the source (DEV) space

In [24]:
! cpdctl asset search --space-id {dev_space_id} --query '*:*' --type-name asset

...
ID                                     Name                         Created                    Description   Type         State       Tags                                                 Size   
ee219910-028a-44ff-93a7-00c80b97af47   credit_risk_regression.csv   2021-08-09T13:30:32.000Z                 data_asset   available   []                                                   694219   
882e7ef8-3b28-41e5-9bd6-8d2c9eb4cf6d   evaluate_model               2021-08-09T13:30:54.000Z                 script       available   []                                                   233   
4baac07a-2f3d-4c7e-9843-45a660ddac1c   gcr-scikit-model             2021-08-12T07:18:17.000Z                 wml_model    available   [cpdctl-demo promoted-asset-2c60c3a8-d461-4a66-b3…   61046   


Export all assets from the source (DEV) space

In [25]:
export = {
    'all_assets': True
}
export_json = json.dumps(export)

! cpdctl asset export start --space-id {dev_space_id} --assets '{export_json}' --name dev-space-all-assets

...
              
ID:        349055bc-2fb2-40d1-accc-29cbe7092744   
Name:      dev-space-all-assets   
Created:   2021-08-16T13:19:05.421Z   
State:     completed   


In [26]:
export_id = '349055bc-2fb2-40d1-accc-29cbe7092744'

In [27]:
dev_space_archive_path = 'dev-space-assets.zip'
! cpdctl asset export download --space-id {dev_space_id} --export-id {export_id} --output-file {dev_space_archive_path}

...
OK
Output written to dev-space-assets.zip


In [28]:
! unzip -l {dev_space_archive_path}

Archive:  dev-space-assets.zip
  Length      Date    Time    Name
---------  ---------- -----   ----
       44  08-16-2021 13:19   deflate.log
      469  08-16-2021 13:19   assettypes/data_asset.json
      476  08-16-2021 13:19   assettypes/omrs_relationship_message.json
      344  08-16-2021 13:19   assettypes/omrs_entity.json
      479  08-16-2021 13:19   assettypes/omrs_relationship.json
      326  08-16-2021 13:19   assettypes/connection.json
      251  08-16-2021 13:19   assettypes/policy_transform.json
      206  08-16-2021 13:19   assettypes/data_flow.json
    16789  08-16-2021 13:19   assettypes/wml_function.json
    43577  08-16-2021 13:19   assettypes/wml_pipeline.json
    40878  08-16-2021 13:19   assettypes/wml_deployment_job_definition.json
     2545  08-16-2021 13:19   assettypes/wml_remote_training_system.json
      546  08-16-2021 13:19   assettypes/connection_credentials.json
      543  08-16-2021 13:19   assettypes/environment.json
      494  08-16-2021 13:19   assett

### 3.2 Create test space and import assets there<a class="anchor" id="part3.2"></a>

Create a new test space

In [71]:
from datetime import datetime

now_dt = datetime.now()
ts = now_dt.strftime("%Y%m%d-%H%M%S")

test_space_name = 'test-space-{}'.format(ts)
result = ! cpdctl space create --name '{test_space_name}' --output json --jmes-query "metadata.id" --raw-output
test_space_id = result.s

print("The new '{}' space ID is: {}".format(test_space_name, test_space_id))

The new 'test-space-20210812-072404' space ID is: 1c6bbc05-e756-4ba7-9ef7-c45c8aaab827


Import assets from the exported archive into test space

In [72]:
result = ! cpdctl asset import start --space-id {test_space_id} --import-file {dev_space_archive_path} --output json --jmes-query "metadata.id" --raw-output
test_import_id = result.s
print("The new import ID is: {}".format(test_import_id))

The new import ID is: 77a1eb30-a792-4f77-96d4-3b2bb2e10ee0


In [73]:
! cpdctl asset import get --space-id {test_space_id} --import-id {test_import_id}

...
              
ID:        77a1eb30-a792-4f77-96d4-3b2bb2e10ee0   
Created:   2021-08-12T07:24:13.200Z   
State:     completed   


List all assets in the test space

In [74]:
! cpdctl asset search --space-id {test_space_id} --query '*:*' --type-name asset

...
ID                                     Name                         Created                    Description   Type         State       Tags   Size   
d709a597-f8d4-4572-a989-f0402b847998   credit_risk_regression.csv   2021-08-12T07:24:17.000Z                 data_asset   available   []     694219   
36e40f7a-155d-45d2-a775-bcbe6b8bb2b5   gcr-scikit-model             2021-08-12T07:24:17.000Z                 wml_model    available   []     61046   
e6a0027f-e4b0-40c8-b3a6-5a0d72951ec6   evaluate_model               2021-08-12T07:24:17.000Z                 script       available   []     233   


Search for the imported asset IDs in the test space

In [75]:
result = ! cpdctl asset search --space-id {test_space_id} --query 'asset.name:gcr-scikit-model' --type-name wml_model --raw-output -j 'results[0].metadata.asset_id' --output json
test_model_id = result.s
print('Model ID: {}'.format(test_model_id))

result = ! cpdctl asset search --space-id {test_space_id} --query 'asset.name:evaluate_model' --type-name script --raw-output -j 'results[0].metadata.asset_id' --output json
test_script_id = result.s
print('Script ID: {}'.format(test_script_id))

result = ! cpdctl asset search --space-id {test_space_id} --query 'asset.name:credit_risk_regression*' --type-name data_asset --raw-output -j 'results[0].metadata.asset_id' --output json
test_data_asset_id = result.s
print('Test data asset ID: {}'.format(test_data_asset_id))

Model ID: 36e40f7a-155d-45d2-a775-bcbe6b8bb2b5
Script ID: e6a0027f-e4b0-40c8-b3a6-5a0d72951ec6
Test data asset ID: d709a597-f8d4-4572-a989-f0402b847998


### 3.3 Run batch deployment job in the test space<a class="anchor" id="part3.3"></a>

Create the script batch deployment in the QA space:

In [76]:
asset = {
    'id': test_script_id
}
asset_json = json.dumps(asset)

hardware_spec = {
    'name': 'S'
}
hardware_spec_json = json.dumps(hardware_spec)

batch_json = '{}'

deployment_name = 'evaluate_model_batch_deployment'

In [77]:
result = ! cpdctl ml deployment create --space-id {test_space_id} --name '{deployment_name}' --asset '{asset_json}' --hardware-spec '{hardware_spec_json}' --batch '{batch_json}' --output json -j "metadata.id" --raw-output
test_batch_deployment_id = result.s
print("ID of the script batch deployment in test space: {}".format(test_batch_deployment_id))

ID of the script batch deployment in test space: a847d548-88e0-4bb8-ac13-c85ac9063fb4


Create a deployment job

In [78]:
deployment_job_name = deployment_name + '_job'

deployment = {
    'id': test_batch_deployment_id
}
deployment_json = json.dumps(deployment)

scoring = {
    "input_data_references": [
      {
        "type": "data_asset",
        "connection": {},
        "location": {
          "href": "/v2/assets/{}?space_id={}".format(test_data_asset_id, test_space_id)
        }
      }
    ],
    "output_data_reference": {
      "type": "data_asset",
      "connection": {},
      "location": {
        "name": "model_evaluation_results.zip"
      }
    }
}
scoring_json = json.dumps(scoring)

In [79]:
result = ! cpdctl ml deployment-job create --space-id {test_space_id} --name '{deployment_job_name}' --deployment '{deployment_json}' --scoring '{scoring_json}' --output json -j "metadata.id" --raw-output
test_batch_deployment_job_id = result.s
print("ID of the script batch deployment job in test space: {}".format(test_batch_deployment_job_id))

ID of the script batch deployment job in test space: 29272d8e-83df-4015-a438-f932f72c9ed6


In [80]:
result = ! cpdctl ml deployment-job get --space-id {test_space_id} --job-id {test_batch_deployment_job_id} --output json
test_batch_deployment_job = json.loads(result.s)
test_batch_job_id = test_batch_deployment_job['entity']['platform_job']['job_id']
test_batch_run_id = test_batch_deployment_job['entity']['platform_job']['run_id']

Wait for job completion

In [81]:
! cpdctl job run wait --job-id {test_batch_job_id} --run-id {test_batch_run_id} --space-id {test_space_id}

...
                  
ID:            3cf4f93b-836d-4590-88ee-4bbb87f1464b   
Name:          job run   
Created:       2021-08-12T07:24:41Z   
Description:      
State:         Completed   
Tags:          []   


You can see the batch deployment log:

In [82]:
! cpdctl job run logs --job-id {test_batch_job_id} --run-id {test_batch_run_id} --space-id {test_space_id}

...
{
  "deployment": {
    "id": "a847d548-88e0-4bb8-ac13-c85ac9063fb4"
  },
  "platform_job": {
    "job_id": "4c4e79bd-686d-4aff-a6e7-3c18cc6e9478",
    "run_id": "3cf4f93b-836d-4590-88ee-4bbb87f1464b"
  },
  "scoring": {
    "input_data_references": [
      {
        "connection": {},
        "location": {
          "href": "/v2/assets/d709a597-f8d4-4572-a989-f0402b847998?space_id=1c6bbc05-e756-4ba7-9ef7-c45c8aaab827"
        },
        "type": "data_asset"
      }
    ],
    "output_data_reference": {
      "connection": {},
      "location": {
        "href": "/v2/assets/a6ada982-b58d-4266-84bd-f5d428db5d23?space_id=1c6bbc05-e756-4ba7-9ef7-c45c8aaab827",
        "name": "model_evaluation_results.zip"
      },
      "type": "data_asset"
    },
    "status": {
      "completed_at": "2021-08-12T07:24:59.947962Z",
      "running_at": "2021-08-12T07:24:58.080822Z",
      "state": "completed"
    }
  }
}



In [83]:
! cpdctl asset search --space-id {test_space_id} --query '*:*' --type-name data_asset

...
ID                                     Name                           Created                    Description   Type         State       Tags   Size   
d709a597-f8d4-4572-a989-f0402b847998   credit_risk_regression.csv     2021-08-12T07:24:17.000Z                 data_asset   available   []     694219   
a6ada982-b58d-4266-84bd-f5d428db5d23   model_evaluation_results.zip   2021-08-12T07:24:59.000Z                 data_asset   available   []     150   


In [84]:
results_asset_id = 'a6ada982-b58d-4266-84bd-f5d428db5d23'

In [85]:
! cpdctl asset get --space-id {test_space_id} --asset-id {results_asset_id}

...
                  
ID:            a6ada982-b58d-4266-84bd-f5d428db5d23   
Name:          model_evaluation_results.zip   
Created:       2021-08-12T07:24:59.000Z   
Description:      
Type:          data_asset   
State:         available   
Tags:          []   
Size:          150   
Attachments:   ID                                     Name                           Description   Type         Mime Type      
               4603ee02-bb01-4d5e-9b1d-4c46ffde9f20   model_evaluation_results.zip                 data_asset   application/zip      
                  


In [88]:
results_attachment_id = '4603ee02-bb01-4d5e-9b1d-4c46ffde9f20'
results_path = 'job_results.zip'

In [89]:
! cpdctl asset attachment download --space-id {test_space_id} --asset-id {results_asset_id} --attachment-id {results_attachment_id} --output-path {results_path}

...
OK
Output written to job_results.zip


In [90]:
! unzip -p {results_path} results.txt

Model validation succeeded!


Clean and delete the test space

In [91]:
! cpdctl ml deployment-job delete --space-id {test_space_id} --job-id {test_batch_deployment_job_id} || echo 'Skipped'
! cpdctl ml deployment delete --space-id {test_space_id} --deployment-id {test_batch_deployment_id} || echo 'Skipped'
! cpdctl space delete --space-id {test_space_id}

...
OK
...
OK
...
OK


## 4. Demo 3: Update assets in the production space <a class="anchor" id="part4"></a>

Before starting with this section, please ensure that you have run the cells in all previous sections: [Section 1](#part1), [Section 2](#part2) and [Section 3](#part3).

### 4.1 Import new asset versions to the production space <a class="anchor" id="part4.1"></a>

Get production space ID

In [92]:
! cpdctl space list

...
ID                                     Name                      Created                    Description   State    Tags   
9001e03a-e3b9-4ddf-88db-063651d735b4   qa-space                  2021-06-21T11:11:22.988Z                 active   []   
715f068a-a8c8-4093-a7f2-15ab111c21e5   prod-space                2021-06-21T11:12:12.958Z                 active   []   
a72bfe88-5fc9-4323-90ad-53d94511d120   fvt-space                 2021-06-28T12:50:33.447Z                 active   []   
cdd90e91-792c-4834-a26a-ac979b998fd8   export-space              2021-07-09T12:02:55.754Z                 active   []   
be32f9b8-702f-444e-9902-754464f39853   jm-space                  2021-07-15T10:40:36.849Z                 active   []   
6237d988-c98f-4f70-acc4-c68f34329105   test-space-cli-eb43d624   2021-07-22T07:49:54.127Z                 active   []   
dd383706-f430-4b1a-8ac8-36084fb821f6   test-export               2021-07-28T17:12:57.575Z                 active   []   
48742238-a5c4-41a6-aaef-fa

In [29]:
prod_space_id = '715f068a-a8c8-4093-a7f2-15ab111c21e5'

In [62]:
! cpdctl asset search --space-id {prod_space_id} --query '*:*' --type-name asset

...
ID                                     Name                                                 Created                    Description   Type                            State       Tags   Size   
e3bd4025-7402-496f-aa27-65f7d4c7b940   evaluate_model_deployment_job                        2021-08-05T14:33:56.000Z                 wml_deployment_job_definition   available   []     0   
4d764f9f-907d-44f9-b9cc-5ecfb52d020c   gcr-scikit-model-batch-deployment-job                2021-08-12T07:52:42.000Z                 wml_deployment_job_definition   available   []     0   
2f4913e3-5e95-4c5a-acdd-4f5a3944c1d8   default job - 2a8b572d-a05f-41f7-8760-ace31058d812   2021-08-16T13:23:34.000Z                 job                             available   []     0   
31837c3a-ee27-4304-8ec0-08db76ff0c18   evaluate_model_deployment_job                        2021-08-16T13:24:48.000Z                 wml_deployment_job_definition   available   []     0   
21604932-7886-473e-9ca2-a40178bbbaf0   model_eva

In [60]:
! cpdctl asset import start --space-id {prod_space_id} --import-file {dev_space_archive_path}

...
              
ID:        796abb00-57e5-4cda-9e06-3079238bd441   
Created:   2021-08-16T13:37:01.536Z   
State:     completed   


### 4.2 Updating the existing model deployment<a class="anchor" id="part4.2"></a>

In [61]:
! cpdctl asset search --space-id {prod_space_id} --query '*:*' --type-name asset

...
ID                                     Name                                                 Created                    Description   Type                            State       Tags   Size   
665b45dc-1519-4b75-9280-d8a8c8e1688a   default job - evaluate_model_deployment              2021-08-05T14:16:41.000Z                 job                             available   []     0   
c51acdf0-f817-4f3b-b52d-9d7eee5ecb68   default job - gcr-scikit-model-batch-deployment      2021-08-12T07:51:23.000Z                 job                             available   []     0   
8e6fee58-a0c2-4f55-a23d-4cc763a3a6af   gcr-scikit-model-batch-deployment-job                2021-08-12T07:52:43.000Z                 job                             available   []     0   
3e6b55dd-ba62-4a7d-8cee-400d74b44d43   job run                                              2021-08-12T07:52:45.000Z                 job_run                         available   []     0   
fb10ee17-f047-4bcc-bfd9-574e22947f0d   evaluate_

In [63]:
prod_model_id = 'fbbb6362-a6d7-4806-9fa3-34e90b0be566'

In [35]:
! cpdctl ml model list-revisions --space-id {prod_space_id} --model-id {prod_model_id}

...
ID                                     Name               Created                    Revision   Tags   
fbbb6362-a6d7-4806-9fa3-34e90b0be566   gcr-scikit-model   2021-08-06T16:27:07.748Z   4          []   
fbbb6362-a6d7-4806-9fa3-34e90b0be566   gcr-scikit-model   2021-08-06T16:27:07.748Z   3          []   
fbbb6362-a6d7-4806-9fa3-34e90b0be566   gcr-scikit-model   2021-08-06T16:27:07.748Z   2          []   
fbbb6362-a6d7-4806-9fa3-34e90b0be566   gcr-scikit-model   2021-08-06T16:27:07.748Z   1          []   


While waiting for fix for the issue [#2075](https://github.ibm.com/AILifecycle/tracker/issues/2075) in `cpdctl` the revision creation date is invalid (it is the model creation time).

In [36]:
prod_model_rev = "4"

In [37]:
 ! cpdctl ml deployment list --space-id {prod_space_id}

...
ID                                     Name                                Created                    State   Tags   
ac34fbb1-6349-4347-b9cd-bf2d9daaccb3   gcr-scikit-model-batch-deployment   2021-08-12T07:51:22.061Z   ready   []   


In [38]:
prod_model_batch_deployment_id = 'ac34fbb1-6349-4347-b9cd-bf2d9daaccb3'

In [39]:
print('Deployed model revision:')
! cpdctl ml deployment get --space-id {prod_space_id} --deployment-id {prod_model_batch_deployment_id} --output json -j 'entity.asset'

Deployed model revision:
{
  "id": "fbbb6362-a6d7-4806-9fa3-34e90b0be566",
  "rev": "3"
}


Patch the batch deployment to use the latest revision

In [40]:
asset = {
  "id": prod_model_id,
  "rev": prod_model_rev
}
asset_json = json.dumps(asset)

! cpdctl ml deployment update --space-id {prod_space_id} --deployment-id {prod_model_batch_deployment_id} --asset '{asset_json}'

...
              
ID:        ac34fbb1-6349-4347-b9cd-bf2d9daaccb3   
Name:      gcr-scikit-model-batch-deployment   
Created:   2021-08-12T07:51:22.061Z   
State:     ready   
Tags:      []   


In [41]:
print('Updated deployed model revision:')
! cpdctl ml deployment get --space-id {prod_space_id} --deployment-id {prod_model_batch_deployment_id} --output json -j 'entity.asset'

Updated deployed model revision:
{
  "id": "fbbb6362-a6d7-4806-9fa3-34e90b0be566",
  "rev": "4"
}


### 4.3 Run production model deployment validation<a class="anchor" id="part4.3"></a>

In [44]:
! cpdctl asset search --space-id {prod_space_id} --query '*:*' --type-name script

...
ID                                     Name             Created                    Description   Type     State       Tags   Size   
e1438649-a1d4-4475-bc82-42ff277cb8d4   evaluate_model   2021-08-16T13:20:56.000Z                 script   available   []     233   


In [45]:
prod_evaluation_script_id = 'e1438649-a1d4-4475-bc82-42ff277cb8d4'

In [64]:
! cpdctl job list --space-id {prod_space_id}

...
ID                                     Name                                                 Description   Tags   Version   
6448850d-1593-464a-b37a-b192655fbc87   evaluate_model_deployment_job                                      []     0   
83e79f30-3500-4d2e-86e7-d05cd03918d6   default job - 84961f48-b0b5-4f84-8e07-60b7b6ddab6e                 []     0   
2f4913e3-5e95-4c5a-acdd-4f5a3944c1d8   default job - 2a8b572d-a05f-41f7-8760-ace31058d812                 []     0   
8e6fee58-a0c2-4f55-a23d-4cc763a3a6af   gcr-scikit-model-batch-deployment-job                              []     0   
c51acdf0-f817-4f3b-b52d-9d7eee5ecb68   default job - gcr-scikit-model-batch-deployment                    []     0   
665b45dc-1519-4b75-9280-d8a8c8e1688a   default job - evaluate_model_deployment                            []     0   


In [65]:
evaluate_model_job_id = '6448850d-1593-464a-b37a-b192655fbc87'

In [66]:
run_json = json.dumps({})
! cpdctl job run create --space-id {prod_space_id} --job-id {evaluate_model_job_id} --job-run '{run_json}'

...
{
  "deployment": {
    "id": "84961f48-b0b5-4f84-8e07-60b7b6ddab6e"
  },
  "hardware_spec": {
    "id": "f3ebac7d-0a75-410c-8b48-a931428cc4c5"
  },
  "platform_job": {
    "job_id": "6448850d-1593-464a-b37a-b192655fbc87",
    "run_id": "cf5065b7-21e7-4bb9-9f40-10c34fb0bb76"
  },
  "scoring": {
    "input_data_references": [
      {
        "connection": {},
        "location": {
          "href": "/v2/assets/3a9df207-58b0-4311-b96b-6e85c36d668d?space_id=715f068a-a8c8-4093-a7f2-15ab111c21e5"
        },
        "type": "data_asset"
      }
    ],
    "output_data_reference": {
      "connection": {},
      "location": {
        "href": "/v2/assets/e6aab593-09a9-4f97-97fc-f2e8919c1697?space_id=715f068a-a8c8-4093-a7f2-15ab111c21e5"
      },
      "type": "data_asset"
    },
    "status": {
      "completed_at": "2021-08-16T13:41:15.132743Z",
      "failure": {
        "trace": "709c75ef840ef018730c66f77da44c27",
        "errors": [
          {
            "code": "invalid_input_data",

In [67]:
results_asset_id = 'e6aab593-09a9-4f97-97fc-f2e8919c1697'

In [68]:
! cpdctl asset get --space-id {prod_space_id} --asset-id {results_asset_id}

...
                  
ID:            e6aab593-09a9-4f97-97fc-f2e8919c1697   
Name:          model_evaluation_results.zip   
Created:       2021-08-16T13:30:13.000Z   
Description:      
Type:          data_asset   
State:         available   
Tags:          []   
Size:          150   
Attachments:   ID                                     Name                           Description   Type         Mime Type      
               3c12bd8a-789c-4db5-866f-85a0d1ed6ca5   model_evaluation_results.zip                 data_asset   application/zip      
                  


In [69]:
results_attachment_id = '3c12bd8a-789c-4db5-866f-85a0d1ed6ca5'
results_path = 'prod_job_results.zip'

In [70]:
! cpdctl asset attachment download --space-id {prod_space_id} --asset-id {results_asset_id} --attachment-id {results_attachment_id} --output-path {results_path}

...
OK
Output written to prod_job_results.zip


In [71]:
! unzip -p {results_path} results.txt

Model validation succeeded!


Update the **existing** script batch deployment in the production space:

In [118]:
deployment_name = 'evaluate_model_deployment'
jmes_query = 'resources[0].metadata.id'
result = ! cpdctl ml deployment list --space-id {prod_space_id} --name {deployment_name} --output json --jmes-query '{jmes_query}' --raw-output
prod_deployment_id = result.s
print('Existing production deployment ID: {}'.format(prod_deployment_id))

Existing production deployment ID: b15e8216-f362-4e58-9d93-5c18a128c68d


Update the deployed asset with the newly created script

In [84]:
asset = {
    'id': prod_script_id
}
asset_json = json.dumps(asset)

In [85]:
! cpdctl ml deployment update --space-id {prod_space_id} --deployment-id {prod_deployment_id} --asset '{asset_json}'

...
              
ID:        25e17b86-9deb-4950-920d-47ba1fc781c1   
Name:      script_batch_deployment   
Created:   2021-04-30T12:39:28.441Z   
State:     ready   
Tags:      []   


Get the deployment job

In [119]:
jmes_query = 'resources[0].entity.platform_job.job_id'
result = ! cpdctl ml deployment-job list --deployment-id {prod_deployment_id} --space-id {prod_space_id} --output json --jmes-query '{jmes_query}' --raw-output
prod_job_id = result.s
print('Production job ID: {}'.format(prod_job_id))

Production job ID: f8ce0da0-bce4-47b5-91eb-c9ac2af3cce7


In [180]:
run = '{}'
jmes_query = 'metadata.asset_id'
result = ! cpdctl job run create --space-id {prod_space_id} --job-id {prod_job_id} --job-run '{run}' --output json --jmes-query '{jmes_query}' --raw-output
prod_run_id = result.s
print('ID of the job run in production space: {}'.format(prod_run_id))

ID of the job run in production space: b558937d-66f1-4d7d-8182-aa1c3ce49cf9


Wait for job completion

In [181]:
! cpdctl job run wait --job-id {prod_job_id} --run-id {prod_run_id} --space-id {prod_space_id}

...
                  
ID:            b558937d-66f1-4d7d-8182-aa1c3ce49cf9   
Name:          job run   
Created:       2021-05-21T15:03:32Z   
Description:      
State:         Completed   
Tags:          []   


You can see the batch deployment log:

In [182]:
! cpdctl job run logs --job-id {prod_job_id} --run-id {prod_run_id} --space-id {prod_space_id}

...
{
  "deployment": {
    "id": "865c5d4b-3439-438a-a2a0-48305237f46d"
  },
  "hardware_spec": {
    "id": "f3ebac7d-0a75-410c-8b48-a931428cc4c5"
  },
  "platform_job": {
    "job_id": "2bb4ef91-cd73-4748-a10a-6316b2a71550",
    "run_id": "b558937d-66f1-4d7d-8182-aa1c3ce49cf9"
  },
  "scoring": {
    "input_data_references": [
      {
        "connection": {},
        "location": {
          "href": "/v2/assets/093424a6-c966-447e-8552-7f2d991f9a76?space_id=f67d7982-7dd0-4ed3-9161-503145f4e0ae"
        },
        "type": "data_asset"
      }
    ],
    "output_data_reference": {
      "connection": {},
      "location": {
        "href": "/v2/assets/ba81206c-6f95-4200-b31d-727ba94b85fb?space_id=f67d7982-7dd0-4ed3-9161-503145f4e0ae"
      },
      "type": "data_asset"
    },
    "status": {
      "completed_at": "2021-05-21T15:04:16.176386Z",
      "message": {
        "level": "warning",
        "text": "The directory pointed by the environment variable BATCH_OUTPUT_DIR is empty, skip

## 5. Cleanup <a class="anchor" id="part5"></a>

Delete QA space

In [227]:
! cpdctl space delete --space-id {qa_space_id} --context cpd

...
OK


### Author

Rafał Bigaj, System Architect with long successful record of building and leading teams. Broad and practical knowledge in the area of cloud computing, machine learning and distributed systems development. 

Copyright © 2020 IBM. This notebook and its source code are released under the terms of the MIT License.